In [5]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime

In [9]:
df_nov=pd.read_excel(r"..\Data\LineC_SessionData-individual rats_SampleData.xlsx")
df_march=pd.read_excel(r"..\Data\Snifftime data_March15.2019_example_withIndividualSniffs.xlsx")
df_sept=pd.read_excel(r"..\Data\AutomatedCage_Sept2023-24_raw data.xlsx")

In [11]:
# Select and rename columns, add empty columns, and calculate new columns for df_march
df_march_mod = df_march.rename(columns={
    'RAT_NAME': 'Rat_Name',
    'ID_EVALUATION_SESSION': 'Evaluation_Session_ID',
    'age': 'Age',
    'RUN': 'Run',
    'HOLE': 'Hole',
    'Sniff 1': 'Sniff_Time',
    'tblRAT_SESSION.SniffThreshold': 'Threshold_Time'
})

# Keep only the required columns (including those to be created)
columns_needed = [
    'Rat_Name',
    'Evaluation_Session_ID',
    'Gender',
    'Age',
    'Weight',
    'Run',
    'Hole',
    'Sniff_Time',
    'Threshold_Time',
    'Result_Of_Rat',
    'Lab_Test'
]

# Add empty columns for Gender and Weight
df_march_mod['Gender'] = ''
df_march_mod['Weight'] = ''
df_march_mod['Sniff_Time'] = df_march_mod['Sniff_Time'].fillna(0)

# Calculate Result_Of_Rat: 1 if Sniff_Time > Threshold_Time, else 0
df_march_mod['Result_Of_Rat'] = (df_march_mod['Sniff_Time'] > df_march_mod['Threshold_Time']).astype(int)

# Calculate Lab_Test: 1 if ID_BL_DOTS > 1, else 0 if == 1
df_march_mod['Lab_Test'] = df_march['ID_BL_DOTS'].apply(lambda x: 1 if x > 1 else (0 if x == 1 else None))

# Reorder columns to match the requested order
df_march_mod = df_march_mod[columns_needed]

# Display the first few rows to check
df_march_mod.head()

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Result_Of_Rat,Lab_Test
0,Daudi,14097,,2.38,,A,1,0.0,2500,0,0
1,Daudi,14097,,2.38,,A,2,3831.0,2500,1,1
2,Daudi,14097,,2.38,,A,3,0.0,2500,0,0
3,Daudi,14097,,2.38,,A,4,0.0,2500,0,0
4,Daudi,14097,,2.38,,A,5,0.0,2500,0,0


In [12]:
# Create FP, FN, TP, TN columns based on Lab_Test (ground truth) and Result_Of_Rat (prediction)
df_march_mod['TP'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['TN'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)
df_march_mod['FP'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['FN'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)

df_march_mod.head()

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Result_Of_Rat,Lab_Test,TP,TN,FP,FN
0,Daudi,14097,,2.38,,A,1,0.0,2500,0,0,0,1,0,0
1,Daudi,14097,,2.38,,A,2,3831.0,2500,1,1,1,0,0,0
2,Daudi,14097,,2.38,,A,3,0.0,2500,0,0,0,1,0,0
3,Daudi,14097,,2.38,,A,4,0.0,2500,0,0,0,1,0,0
4,Daudi,14097,,2.38,,A,5,0.0,2500,0,0,0,1,0,0
